- https://github.com/ksopyla/awesome-nlp-polish
- Sentiment: https://pypi.org/project/sentimentpl/
- Auto correct: https://github.com/filyp/autocorrect
- other: https://github.com/sdadas/polish-nlp-resources
- papers: https://homados.ipipan.waw.pl/?page_id=93

In [14]:
import pandas as pd
import numpy as np
import spacy
from sentimentpl.models import SentimentPLModel
from autocorrect import Speller
from transformers import HerbertTokenizer, RobertaModel

import re

from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=7,progress_bar=True)

import unicodedata

from parallelbar import progress_map


INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
import polyglot
from polyglot.text import Text, Word

In [3]:
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer#for word embedding

import gensim
from gensim.models import Word2Vec

In [4]:
nlp_core = spacy.load("pl_core_news_lg")

In [5]:
model = SentimentPLModel(from_pretrained='latest')

In [6]:
spell = Speller('pl')

In [7]:
# tokenizer_herbert = HerbertTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
# model_roberta = RobertaModel.from_pretrained("allegro/herbert-klej-cased-v1")

In [8]:
# nlp_pl = spacy.load('pl_spacy_model') # or spacy.load('pl_spacy_model_morfeusz')

In [9]:
def clean_przyp(txt):
    if txt != txt:
        return np.nan
    
    txt_out = txt
    
    if "przyp. Demagog" in txt:
        txt_out = (txt_out
                   .replace('(','').replace(')','')
                   .replace(' – przyp. Demagog','')
                   .replace('- red.', ''))
    if "(…)" in txt:
        txt_out =  txt_out.replace('(…)','')
    if "(...)" in txt:
        txt_out =  txt_out.replace('(...)','')
    if "[" in txt:
        txt_out = txt_out.replace('[','').replace(']','')
        
    txt_out = re.sub("@[A-Za-z0-9]+","",txt_out) #Remove @ sign
    txt_out = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", txt_out) #Remove http links
    
    txt_out = unicodedata.normalize("NFKD", txt_out) #cleaning html
    
    txt_out = txt_out.replace(';', '.').replace('  ', ' ')
    
    return txt_out

In [10]:
def extract_features(txt):
    
    doc = nlp_core(txt)
    
    out_dict = {}
    
    lemmas_list = []
    tokens_list = []
    sentiments_list = []
    embeddings_list = []

    error_n = 0

    adj_n = 0
    adv_n = 0
    noun_n = 0
    ent_n = 0
    
    upper_s_n = 0
    upper_f_n = sum(map(str.isupper, txt.split()))
    
    e_mark_n = txt.count('!')
    q_mark_n = txt.count('?')
    
    upper_n = sum(1 for c in txt if c.isupper())
    
    out_dict['sentiment_all'] = model(doc.text).item()
    
    for i, sent in enumerate(doc.sents):
        s = model(sent.text).item()
        sentiments_list.append(s)
    
    out_dict['sentiment_avg'] = np.mean(sentiments_list)

    txt_pos = []
    for token in doc:
        if token.pos_ not in ['SPACE', 'PUNCT']:
            lemmas_list.append(token.lemma_)
            tokens_list.append(token.text)
            corrected = spell(token.text)
            if corrected != token.text:
                error_n += 1

        if token.pos_ == 'ADJ': 
            adj_n += 1
        elif token.pos_ == 'ADV':
            adv_n += 1
        elif token.pos_ == 'NOUN':
            noun_n += 1
        
        if token.text[0].isupper():
            upper_s_n += 1
        
        txt_pos.append(token.pos_)
        

    for ent in doc.ents:
        ent_n += 1

    tokens_list = list(set(tokens_list))
    lemmas_list = list(set(lemmas_list))

    out_dict['uniq_words'] = len(tokens_list)
    out_dict['uniq_lemm'] =  len(lemmas_list)
    out_dict['err'] =  error_n
    out_dict['net'] = ent_n
    out_dict['ADJ'] = adj_n/len(tokens_list)
    out_dict['ADV'] = adv_n/len(tokens_list)
    out_dict['NOUN'] = noun_n/len(tokens_list)
    
    out_dict['words_start_upper'] = upper_s_n
    out_dict['words_full_upper'] = upper_f_n
    out_dict['exclamation_marks'] = e_mark_n
    out_dict['question_marks'] = q_mark_n
    
    out_dict['upper_letters'] = upper_n
    out_dict['chars'] = len(txt)

    out_dict['TEXT_POS'] = txt_pos
    
    return out_dict

# CZ data

In [11]:
df_pl_cz = pd.read_csv('../datasets/demagog_nlp_cz/converted-exp-PL.tsv', sep='\t')
df_pl_cz.shape

(2835, 8)

In [12]:
df_pl_cz.head()

politicianID              name                   party  statementID  \
0             1  Paweł Szałamacha  Prawo i Sprawiedliwość            1   
1             1  Paweł Szałamacha  Prawo i Sprawiedliwość            2   
2             1  Paweł Szałamacha  Prawo i Sprawiedliwość            3   
3             1  Paweł Szałamacha  Prawo i Sprawiedliwość            4   
4             1  Paweł Szałamacha  Prawo i Sprawiedliwość            5   

                                       statementText statementState  \
0  Pamiętamy, że w latach 2006-2007, kiedy rządzi...           TRUE   
1  Jeżeli patrzymy na dane dla OECD, to Polska je...           TRUE   
2  Po kilkuletnim okresie dodatniego przyrostu na...           TRUE   
3  Jeszcze dwa dni wcześniej Bank Światowy podwyż...           TRUE   
4  Jeszcze dzień wcześniej Międzynarodowy Fundusz...           TRUE   

                                statementExplanClean  \
0  Według danych Banku Światowego wzrost polskieg...   
1  Według danych z OECD, Polska znajduje się poni...   
2  Liczba urodzonych dzieci w Polsce na 1000 mies...   
3  Według danych Banku Światowego zawartych w ana...   
4  W dniu 13 stycznia 2016 r. na stronie Międzyna...   

                                     statementExplan  
0  <p><span>Według danych </span><a href="http://...  
1  <p><span>Według </span><a href="http://www.oec...  
2  <p><a href="http://www.indexmundi.com/g/g.aspx...  
3  <p><span>Według danych Banku Światowego zawart...  
4  <p><span>W dniu 13 stycznia 2016 r. na stronie...

In [13]:
df_pl_cz['text_clean'] = df_pl_cz['statementText'].apply(lambda x: clean_przyp(x))

In [15]:
tasks = df_pl_cz['text_clean'].values.tolist()
result = progress_map(extract_features, tasks, n_cpu=7, chunk_size=1, core_progress=True)

Core 1:   0%|          | 0/405 [00:00<?, ?it/s]

Core 2:   0%|          | 0/405 [00:00<?, ?it/s]

Core 3:   0%|          | 0/405 [00:00<?, ?it/s]

Core 4:   0%|          | 0/405 [00:00<?, ?it/s]

Core 5:   0%|          | 0/405 [00:00<?, ?it/s]

Core 6:   0%|          | 0/405 [00:00<?, ?it/s]

Core 7:   0%|          | 0/405 [00:00<?, ?it/s]

In [16]:
df_pl_cz['raw_dict'] = result 

In [17]:
df_clean = (
    df_pl_cz.join( 
        df_pl_cz['raw_dict']
        .apply(pd.Series)
    ).drop('raw_dict', axis=1)
)

In [18]:
df_clean['TEXT_POS'] = df_clean['TEXT_POS'].str.join(" ")

In [21]:
df_clean.to_csv('../datasets/ready2use/fake_news_features_cz.csv', sep=';', index=False, encoding='utf8')

# Demagog

In [11]:
df = pd.read_csv('../datasets/scrapped/demagog.csv', sep=';')
df.shape

(4924, 4)

In [12]:
df = df.dropna()
df = df[df['text'].str.len() > 0 ]
df.shape

(4922, 4)

In [13]:
df['text_clean'] = df['text'].apply(lambda x: clean_przyp(x))

In [14]:
df = df[df['text_clean'].str.len() > 1 ]
df.shape

(4917, 5)

In [33]:
tasks = df['text_clean'].values.tolist()
result = progress_map(extract_features, tasks, n_cpu=7, chunk_size=1, core_progress=True)

Core 1:   0%|          | 0/703 [00:00<?, ?it/s]

Core 2:   0%|          | 0/703 [00:00<?, ?it/s]

Core 3:   0%|          | 0/703 [00:00<?, ?it/s]

Core 4:   0%|          | 0/703 [00:00<?, ?it/s]

Core 5:   0%|          | 0/703 [00:00<?, ?it/s]

Core 6:   0%|          | 0/703 [00:00<?, ?it/s]

Core 7:   0%|          | 0/703 [00:00<?, ?it/s]

In [34]:
df['raw_dict'] = result #df['text_clean'].progress_apply(lambda x: extract_features(x))

In [35]:
df_clean = (
    df.join( 
        df['raw_dict']
        .apply(pd.Series)
#         .rename(columns={'sentiment_lemm' : 'uniq_lemm'}) 
    ).drop('raw_dict', axis=1)
)

In [43]:
df_clean['TEXT_POS'] = df_clean['TEXT_POS'].str.join(" ")

In [44]:
df_clean.to_csv('../datasets/scrapped/demagog_features.csv', sep=';', index=False)

# OKO.press

In [45]:
df_oko_raw = pd.read_csv('../datasets/oko.press/query_result.tsv', sep='\t')

print(df_oko_raw['Id wypowiedzi'].unique().shape)

df_oko = pd.merge(
    df_oko_raw.pivot(index=['Id wypowiedzi'], columns='Nazwa pola danych', values='Wartość pola danych').reset_index(),
    df_oko_raw[~df_oko_raw['Autor Wypowiedzi'].isin(['Link do hasła', 'Nazwa słupka', 'Wesprzyj nas'])] \
        [['Id wypowiedzi', 'Autor Wypowiedzi']].dropna(),
    on='Id wypowiedzi',
    how='left'
)

print(df_oko.shape)

(2869,)
(2869, 9)


In [46]:
df_oko.head()

Id wypowiedzi  sub_date                                     sub_hiperlacze  \
0           1069  20160511  http://www.polskieradio.pl/7/129/Artykul/16180...   
1           1172  20160511                                                NaN   
2           1180  20160511                                                NaN   
3           1261  20160516  http://www.tokfm.pl/Tokfm/1,145400,20083911,mo...   
4           1411  20160512  https://www.wprost.pl/kraj/10006923/Polska-jes...   

                                           sub_napis  \
0  Rządy Tuska to również doprowadzenie do wyzysk...   
1  Absurdy i marnotrawstwo pokazuje najlepiej pro...   
2  Przez 15 lat finansowaliście budowę korwety Ga...   
3  Dzisiaj ponad 65 procent długu państwowego jes...   
4  Polska jest gotowa przyjąć każdego uchodźcę, k...   

  sub_napis_autor_wypowiedzi  \
0                       1067   
1                       1026   
2                       1026   
3                       1257   
4                       1076   

                                         sub_podpis sub_stan_zegara  \
0                     „Sygnały Dnia”, Polskie Radio           falsz   
1                                              Sejm   blisko_prawdy   
2                                              Sejm   blisko_prawdy   
3                                            TOK FM           falsz   
4  Wywiad dla tygodnika „Mclean’s” za: Prezydent.pl           falsz   

                     sub_title_text_after      Autor Wypowiedzi  
0        Fałsz - wycieka kilka razy mniej  Stanisław Piotrowicz  
1         Jest moździerz, nie ma amunicji    Antoni Macierewicz  
2  Niedokończony okręt za miliard złotych    Antoni Macierewicz  
3       Fałsz - pomylił się o 92 miliardy    Mateusz Morawiecki  
4               Fałsz - Duda bez pokrycia          Andrzej Duda

In [47]:
df_oko_fin = df_oko[['sub_napis', 'sub_stan_zegara', 'Autor Wypowiedzi']]

In [48]:
df_oko_fin['text_clean'] = df_oko_fin['sub_napis'].progress_apply(lambda x: clean_przyp(x))








100%|███████████████████████████████████| 2869/2869 [00:00<00:00, 150506.65it/s]
/tmp/ipykernel_45543/3568069148.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oko_fin['text_clean'] = df_oko_fin['sub_napis'].progress_apply(lambda x: clean_przyp(x))


In [49]:
tasks = df_oko_fin['text_clean'].values.tolist()
result = progress_map(extract_features, tasks, n_cpu=7, chunk_size=1, core_progress=True)

Core 1:   0%|          | 0/410 [00:00<?, ?it/s]

Core 2:   0%|          | 0/410 [00:00<?, ?it/s]

Core 3:   0%|          | 0/410 [00:00<?, ?it/s]

Core 4:   0%|          | 0/410 [00:00<?, ?it/s]

Core 5:   0%|          | 0/410 [00:00<?, ?it/s]

Core 6:   0%|          | 0/410 [00:00<?, ?it/s]

Core 7:   0%|          | 0/410 [00:00<?, ?it/s]

In [50]:
df_oko_fin['raw_dict'] = result #df_oko_fin['text_clean'].progress_apply(lambda x: extract_features(x))

/tmp/ipykernel_45543/4155824533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oko_fin['raw_dict'] = result #df_oko_fin['text_clean'].progress_apply(lambda x: extract_features(x))


In [51]:
df_oko_clean = df_oko_fin.join( df_oko_fin['raw_dict'].apply(pd.Series) ).drop('raw_dict', axis=1)

In [52]:
df_oko_clean['TEXT_POS'] = df_oko_clean['TEXT_POS'].str.join(" ")

In [54]:
df_oko_clean.to_csv('../datasets/oko.press/okopress_features.csv', sep=';', index=False)

In [56]:
df_oko_clean.columns

Index(['sub_napis', 'sub_stan_zegara', 'Autor Wypowiedzi', 'text_clean',
       'sentiment_all', 'sentiment_avg', 'uniq_words', 'uniq_lemm', 'err',
       'net', 'ADJ', 'ADV', 'NOUN', 'words_start_upper', 'words_full_upper',
       'exclamation_marks', 'question_marks', 'upper_letters', 'chars',
       'TEXT_POS'],
      dtype='object')